In [3]:
import pandas as pd
import re
from string import punctuation as pun
from chat_word_dict import abbreviations
from textblob import TextBlob
from gensim.parsing.preprocessing import STOPWORDS
import string

In [5]:
data = pd.read_csv('data/raw_data.csv', encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])

In [6]:
data

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [7]:
data.drop(columns=['ids', 'date', 'flag', 'user'], inplace=True)

In [8]:
data

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


PreProcessing:

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# function to clean the text column
def clean_text(text):
    # lower-casing
    text = text.lower()
    # remove urls
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)

    # remove mentions and hash-tages
    mentions_pattern = re.compile(r'@\S+')
    text = mentions_pattern.sub(r'', text)
    hashs_pattern = re.compile(r'#\S+')
    text = hashs_pattern.sub(r'', text)

    # remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # convert abbreviations
    pattern = r'\b(' + '|'.join(re.escape(key) for key in abbreviations.keys()) + r')\b'
    text = re.sub(pattern, lambda match: abbreviations[match.group(0).lower()], text)

    # remove stopwords
    text =  ' '.join(word for word in text.split() if word.lower() not in STOPWORDS)

    # remove emojis
    text = remove_emoji(text)

    return text

Speeling Correction:

In [10]:
#data['text'] = data['text'].apply(lambda x: str(TextBlob(x).correct()))

In [11]:
data['text'] = data['text'].apply(clean_text)

In [13]:
data.to_csv('data/processed_data.csv', index=False)